In [1]:
import calendar
import datetime as dt
from timeit import default_timer as timer

import numpy as np
import pandas as pd

start_time = timer()

In [2]:
df = pd.read_csv('applications data.csv')

In [3]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [4]:
df.date = pd.to_datetime(df.date,format='%Y%m%d')

In [5]:
df.dtypes

record                  int64
date           datetime64[ns]
ssn                     int64
firstname              object
lastname               object
address                object
zip5                    int64
dob                     int64
homephone               int64
fraud_label             int64
dtype: object

In [6]:
df.shape

(1000000, 10)

### Clean frivolous fields

In [7]:
df.loc[df.ssn == 999999999, 'ssn'] = df.loc[df.ssn == 999999999, 'record']
df.loc[df.address == '123 MAIN ST', 'address'] = df.loc[df.address == '123 MAIN ST', 'record']
df.loc[df.homephone == 9999999999, 'homephone'] = df.loc[df.homephone == 9999999999, 'record']
df.loc[df.dob == 19070626, 'dob'] = df.loc[df.dob == 19070626, 'record']

In [8]:
df.ssn = df.ssn.astype(str)
df.zip5 = df.zip5.astype(str)
df.dob = df.dob.astype(str)
df.homephone = df.homephone.astype(str)
df.address = df.address.astype(str)

In [9]:
# add leading 0 to zips

df['zip5'] = df.zip5.apply(lambda x: x if len(x) == 5 else '0'*(5-len(x)) + x)

### Risk table for day of week

In [10]:
df['dow'] = df.date.apply(lambda x: calendar.day_name[x.weekday()])

In [11]:
train_test = df[df.date < '2016-11-01']

In [12]:
# do statistical smoothing
c = 4; nmid = 20; y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

In [13]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,dow,dow_risk
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,1,1797504115,0,Friday,0.014499
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,Friday,0.014499
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0,Friday,0.014499
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0,Friday,0.014499
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,Friday,0.014499


### Create entities

In [14]:
df['name'] = df.firstname + df.lastname
df['fulladdress'] = df.address + df.zip5
df['name_dob'] = df.name + df.dob
df['name_fulladdress'] = df.name + df.fulladdress
df['name_homephone'] = df.name + df.homephone
df['fulladdress_dob'] = df.fulladdress + df.dob
df['fulladdress_homephone'] = df.fulladdress + df.homephone
df['dob_homephone'] = df.dob + df.homephone
df['homephone_name_dob'] = df.homephone + df.name_dob

In [15]:
for field in list(df.iloc[:,np.r_[3:9, 12:15]].columns):
    df['ssn_' + field] = df.ssn + df[field]

In [16]:
attributes = list(df.iloc[:, np.r_[2, 5, 7, 8, 12:30]].columns)

In [17]:
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name',
 'fulladdress',
 'name_dob',
 'name_fulladdress',
 'name_homephone',
 'fulladdress_dob',
 'fulladdress_homephone',
 'dob_homephone',
 'homephone_name_dob',
 'ssn_firstname',
 'ssn_lastname',
 'ssn_address',
 'ssn_zip5',
 'ssn_dob',
 'ssn_homephone',
 'ssn_name',
 'ssn_fulladdress',
 'ssn_name_dob']

### Velocity + Day since

In [18]:
df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.date
df1['check_record'] = df1.record

In [19]:
start = timer()

for entity in attributes:
    
    st = timer()

    df_l = df1[['record', 'date', entity]]
    df_r = df1[['check_record', 'check_date', entity]]
    temp = pd.merge(df_l, df_r, left_on=entity, right_on=entity)

    # day since
    day_since_df = temp[temp.record > temp.check_record][['record', 'date', 'check_date']] \
        .groupby('record')[['date', 'check_date']].last()
    mapper = (day_since_df.date - day_since_df.check_date).dt.days
    final[entity + '_day_since'] = final.record.map(mapper)
    final[entity + '_day_since'].fillna(365, inplace=True)
    print(f'\n{entity}_day_since ---> Done')

    # velocity
    for offset_t in [0, 1, 3, 7, 14, 30]:
        count_day_df = temp[(temp.check_date >= (temp.date - dt.timedelta(offset_t)))
                            & (temp.record >= temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('record')[entity].count()
        final[col_name] = final.record.map(mapper2)

        print(f'{entity}_count_{str(offset_t)} ---> Done')

    print(f'Run time for entity {entity} ----------------- {timer() - st:0.2f}s')

print(f'Total run time: {(timer() - start) / 60:0.2f}min')


ssn_day_since ---> Done
ssn_count_0 ---> Done
ssn_count_1 ---> Done
ssn_count_3 ---> Done
ssn_count_7 ---> Done
ssn_count_14 ---> Done
ssn_count_30 ---> Done
Run time for entity ssn ----------------- 2.22s

address_day_since ---> Done
address_count_0 ---> Done
address_count_1 ---> Done
address_count_3 ---> Done
address_count_7 ---> Done
address_count_14 ---> Done
address_count_30 ---> Done
Run time for entity address ----------------- 2.23s

dob_day_since ---> Done
dob_count_0 ---> Done
dob_count_1 ---> Done
dob_count_3 ---> Done
dob_count_7 ---> Done
dob_count_14 ---> Done
dob_count_30 ---> Done
Run time for entity dob ----------------- 15.34s

homephone_day_since ---> Done
homephone_count_0 ---> Done
homephone_count_1 ---> Done
homephone_count_3 ---> Done
homephone_count_7 ---> Done
homephone_count_14 ---> Done
homephone_count_30 ---> Done
Run time for entity homephone ----------------- 13.08s

name_day_since ---> Done
name_count_0 ---> Done
name_count_1 ---> Done
name_count_3 ---> 

### Relative Velocity

In [20]:
start = timer()
for att in attributes:
    for d in ['0', '1']:
        for dd in ['3', '7', '14', '30']:
            final[att + '_count_' + d + '_by_' + dd] \
                = final[att + '_count_' + d] / (final[att + '_count_' + dd] / float(dd))
print(f'Total run time: {timer() - start:0.2f}s')

Total run time: 2.82s


### Keep desired variables

In [21]:
final.set_index('record', inplace = True)

In [22]:
final = final.iloc[:, np.r_[8, 10, 29:337]]
final.shape

(1000000, 310)

In [23]:
final.head()

,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,ssn_count_30,address_day_since,...,ssn_homephone_count_0_by_3,ssn_homephone_count_0_by_7,ssn_homephone_count_0_by_14,ssn_homephone_count_0_by_30,ssn_homephone_count_1_by_3,ssn_homephone_count_1_by_7,ssn_homephone_count_1_by_14,ssn_homephone_count_1_by_30,ssn_name_count_0_by_3,ssn_name_count_0_by_7
record,,,,,,,,,,,,,,,,,,,,,
1,0,0.014499,365.0,1,1,1,1,1,1,365.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
2,1,0.014499,365.0,1,1,1,1,1,1,365.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
3,0,0.014499,365.0,1,1,1,1,1,1,365.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
4,0,0.014499,365.0,1,1,1,1,1,1,365.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
5,0,0.014499,365.0,1,1,1,1,1,1,365.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0


In [24]:
final.to_csv('vars_308.csv')

In [25]:
print(f'Duration: {(timer() - start_time) / 60: 0.2f} min')

Duration:  3.19 min
